In [1]:
include("LiPoSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5

using Statistics

In [2]:
dodeca_10_states = ["D"*string(n) for n=1:10];
basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

evol_data_file_name = "DATA/ALL_GAMMAS_B4_D10.h5"

"DATA/ALL_GAMMAS_B4_D10.h5"

In [6]:

function read_H_Js_from_hdf5(file_path::String, gamma_group::String)
    H = []
    Js = []
    
    h5open(file_path, "r") do fid
        # Navigate to the gamma group
        γ_group = fid[gamma_group]
        
        # Read H and J1 to J4 as they are, without slicing or indexing
        H = read(γ_group["H"])
        Js = [read(γ_group["J$i"]) for i in 1:4]
    end
    
    return H, Js
end

read_H_Js_from_hdf5 (generic function with 1 method)

In [3]:
models_file = "MODEL_LindbladFrob_Jump_NLOPT_LD_SLSQP2024-Oct-04_at_11-24.h5"

"MODEL_LindbladFrob_Jump_NLOPT_LD_SLSQP2024-Oct-04_at_11-24.h5"

In [7]:
H, Js = read_H_Js_from_hdf5(models_file, "0.079477")

(ComplexF64[12.561044754034528 + 0.0im 4.675745626956162e-5 - 4.345168871252416e-5im; 4.675745626956162e-5 + 4.345168871252416e-5im -12.561044754034528 + 0.0im], Matrix{ComplexF64}[[0.0 + 0.0im 0.30848168203649334 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 - 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.0 + 0.0im]])

In [25]:
println(" Initial state of Spin boson data")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133"] #, "7.9477", "25.133", "79.477", "251.33"]

non_physical_time_data_file = "SB_simulation_init_states.h5"

init_states_by_gamma = []
init_times_by_gamma = []

for γᵢ in γ

    println()
    println("γ =  "*γᵢ)

    H, Js = read_H_Js_from_hdf5(models_file, γᵢ)
    H = DenseOperator(basis,H)
    Js = [DenseOperator(basis,j) for j in Js]

    for state in test_states # loop over initial states
        println()
        
        print(state*" ")

        tₛ, ρₛ = LiPoSID.read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₒ = DenseOperator(basis, ρₛ[1]); 
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)
        tₒ = tᵗˢᵗ[1]
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        print("Length: ")

        for i in 1:30   
            print(string(i)*"|")
            tˢⁱᵈ, ρˢⁱᵈ = timeevolution.master(tᵗˢᵗ, ρₒ, H, Js)
            ρₒ  = ρˢⁱᵈ[end]
            tᵗˢᵗ = tᵗˢᵗ .+ tˢⁱᵈ[end]
            if abs(abs(tr(ρₒ.data)) - 1) > 1e-12
                print(abs(tr(ρₒ.data)))
            end
        end

        #h5open(init_states_data_file,"cw") do fid
        #    γ_group = open_group(fid, γᵢ) # open gamma coupling group
        #    init_state_group = create_group(γ_group, state) # create initial state group
        #    init_state_group["non_physical_time"] = t_non_phys
        #end
    
    
    end

end

 Initial state of Spin boson data

γ =  0.079477

D1 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D2 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D3 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D4 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D5 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D6 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D7 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D8 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D9 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
D10 Length: 1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|
γ =  0.25133

D1 Le

In [8]:
for i in 1:100
    print(i)
end

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100